In [ ]:
%reset

In [ ]:
from os import listdir
from os.path import exists
from importlib import reload
import numpy as np
import pandas as pd
import pyxdf
import mne
from utils import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
import time
import datetime
from datetime import datetime, timezone
import pickle
import plotly.express as px

print('Imports done...')

In [ ]:
#data_path = 'C:/Users/tumfart/Code/github/master-thesis/data/'
data_path = 'C:/Users/peter/Google Drive/measurements/eeg/'
subjects = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07' , 'A08', 'A09', 'A10']
# = 'A03'
paradigm = 'paradigm' # 'eye', 'paradigm'
plot = False
mne.set_log_level('WARNING')
# Create path list for each subject:
paths = [str(data_path + subject + '/' + paradigm) for subject in subjects]

In [ ]:
# Read all .npy coefficients:
start = time.time()
A = []
for subject, path in zip(subjects, paths):
    file_names = [f for f in listdir(path) if 'regr_coeff.npy' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    A.append(np.load(file))

In [ ]:
# Calculate mean for each channel and for each timestamp:
n_chan, n_coeff, n_ts = A[0].shape
means = np.empty((n_chan, n_coeff, n_ts))
uppers = np.empty((n_chan, n_coeff, n_ts))
lowers = np.empty((n_chan, n_coeff, n_ts))
means[:], uppers[:], lowers[:] = np.nan, np.nan, np.nan

confidence = 0.95
start = time.time()
for ts in range(n_ts):
    print(f'{ts}/{n_ts}', end='\r')
    for coeff in range(n_coeff):
        for chan in range(n_chan):
            a = [A[i][chan, coeff, ts] for i in range(len(subjects))] # Get list of all subject for a specific channel, coefficient and timestamp

            # Calculate mean:
            # Bootstrapping for confidence interval:
            values = [np.random.choice(a, size=len(a),replace=True).mean() for i in range(500)]
            means[chan,coeff,ts] = np.array(values).mean()
            lowers[chan,coeff,ts], uppers[chan,coeff,ts] = np.percentile(values,[100*(1-confidence)/2,100*(1-(1-confidence)/2)])

print(f'Bootstrapping, took me {round(time.time() - start)} seconds...')

In [ ]:
# Save bootstrapping matrices:
np.save('regr_coeff_global_means.npy', means)    # .npy extension is added if not given
np.save('regr_coeff_global_lowers.npy', lowers)
np.save('regr_coeff_global_uppers.npy', uppers)